In [0]:
filePath = "wasbs://" + blob_container + "@" + storage_account_name + ".blob.core.windows.net"
container_url = "wasbs://" + blob_container + "@" + storage_account_name + ".blob.core.windows.net"

In [0]:
filePath = "wasbs://" + blob_container + "@" + storage_account_name + ".blob.core.windows.net/"

In [0]:
spark.conf.set("fs.azure.createRemoteFileSystemDuringInitialization", "true")

In [0]:
dbutils.fs.ls(filePath + 'nyc_uber')

Out[99]: [FileInfo(path='wasbs://[REDACTED]@[REDACTED].blob.core.windows.net/nyc_uber/uber-raw-data-apr14.csv', name='uber-raw-data-apr14.csv', size=26110064, modificationTime=1675420670000),
 FileInfo(path='wasbs://[REDACTED]@[REDACTED].blob.core.windows.net/nyc_uber/uber-raw-data-aug14.csv', name='uber-raw-data-aug14.csv', size=38324585, modificationTime=1675420695000),
 FileInfo(path='wasbs://[REDACTED]@[REDACTED].blob.core.windows.net/nyc_uber/uber-raw-data-jul14.csv', name='uber-raw-data-jul14.csv', size=36870544, modificationTime=1675420694000),
 FileInfo(path='wasbs://[REDACTED]@[REDACTED].blob.core.windows.net/nyc_uber/uber-raw-data-jun14.csv', name='uber-raw-data-jun14.csv', size=30695600, modificationTime=1675420697000),
 FileInfo(path='wasbs://[REDACTED]@[REDACTED].blob.core.windows.net/nyc_uber/uber-raw-data-may14.csv', name='uber-raw-data-may14.csv', size=30174084, modificationTime=1675420681000),
 FileInfo(path='wasbs://[REDACTED]@[REDACTED].blob.core.windows.net/nyc_uber

In [0]:
dbutils.fs.mount(
    source = container_url,
    mount_point = "/mnt/home",
    extra_configs = {'fs.azure.account.key.' + storage_account_name + '.blob.core.windows.net':
        dbutils.secrets.get(scope = "scope_cgbigdatalab01", key = "storageAccount01key")})

Out[103]: True

In [0]:
home = '/mnt/home/nyc_uber'
dbutils.fs.ls(home)

Out[104]: [FileInfo(path='dbfs:/mnt/home/nyc_uber/uber-raw-data-apr14.csv', name='uber-raw-data-apr14.csv', size=26110064, modificationTime=1675420670000),
 FileInfo(path='dbfs:/mnt/home/nyc_uber/uber-raw-data-aug14.csv', name='uber-raw-data-aug14.csv', size=38324585, modificationTime=1675420695000),
 FileInfo(path='dbfs:/mnt/home/nyc_uber/uber-raw-data-jul14.csv', name='uber-raw-data-jul14.csv', size=36870544, modificationTime=1675420694000),
 FileInfo(path='dbfs:/mnt/home/nyc_uber/uber-raw-data-jun14.csv', name='uber-raw-data-jun14.csv', size=30695600, modificationTime=1675420697000),
 FileInfo(path='dbfs:/mnt/home/nyc_uber/uber-raw-data-may14.csv', name='uber-raw-data-may14.csv', size=30174084, modificationTime=1675420681000),
 FileInfo(path='dbfs:/mnt/home/nyc_uber/uber-raw-data-sep14.csv', name='uber-raw-data-sep14.csv', size=47540750, modificationTime=1675420703000)]

In [0]:
%sh 
ls /dbfs/mnt/home/nyc_uber

uber-raw-data-apr14.csv
uber-raw-data-aug14.csv
uber-raw-data-jul14.csv
uber-raw-data-jun14.csv
uber-raw-data-may14.csv
uber-raw-data-sep14.csv


In [0]:
uber = '/mnt/home/nyc_uber/'
df = spark.read.csv(uber, inferSchema = False, header = True) 

In [0]:
from pyspark.sql.functions import when
df2 = df.withColumn("Uber_base", when(df.Base == "B02512","Unter")
 .when(df.Base == "B02598", "Hinter")
 .when(df.Base == "B02617", "Weiter")
 .when(df.Base == "B02682", "Schmecken")
 .when(df.Base == "B02764", "Danach-NY")
 .when(df.Base == "B02765", "Grun")
 .when(df.Base == "B02835", "Dreist")
 .when(df.Base == "B02836", "Drinnen")
 .otherwise(df.Base))

df2.show(10)

+----------------+-------+--------+------+---------+
|       Date/Time|    Lat|     Lon|  Base|Uber_base|
+----------------+-------+--------+------+---------+
|9/1/2014 0:01:00|40.2201|-74.0021|B02512|    Unter|
|9/1/2014 0:01:00|  40.75|-74.0027|B02512|    Unter|
|9/1/2014 0:03:00|40.7559|-73.9864|B02512|    Unter|
|9/1/2014 0:06:00| 40.745|-73.9889|B02512|    Unter|
|9/1/2014 0:11:00|40.8145|-73.9444|B02512|    Unter|
|9/1/2014 0:12:00|40.6735|-73.9918|B02512|    Unter|
|9/1/2014 0:15:00|40.7471|-73.6472|B02512|    Unter|
|9/1/2014 0:16:00|40.6613|-74.2691|B02512|    Unter|
|9/1/2014 0:32:00|40.3745|-73.9999|B02512|    Unter|
|9/1/2014 0:33:00|40.7633|-73.9773|B02512|    Unter|
+----------------+-------+--------+------+---------+
only showing top 10 rows



In [0]:
df2.createOrReplaceTempView('df_2')

In [0]:
import pyspark.sql.functions as F
df_agg = df2.withColumn("date_time_dt", F.to_timestamp(F.col("Date/Time"),"M/d/yyyy H:mm:ss")) 

df_agg.show(1)

+----------------+-------+--------+------+---------+-------------------+
|       Date/Time|    Lat|     Lon|  Base|Uber_base|       date_time_dt|
+----------------+-------+--------+------+---------+-------------------+
|9/1/2014 0:01:00|40.2201|-74.0021|B02512|    Unter|2014-09-01 00:01:00|
+----------------+-------+--------+------+---------+-------------------+
only showing top 1 row



In [0]:
df_agg.createOrReplaceTempView('df_agg')

In [0]:
import pyspark.sql.functions as F
def check_symb(column):
    cnt = 0
    for sym in [' ', ',', ';', '"', "'"]:
        cnt += 1
        if sym in column:  
            return column.replace(sym, '.')   
        else:
            if cnt == 5:
                cnt = 0
                return column
            
sym_drop = F.udf(check_symb)

In [0]:
df_1 = df\
    .withColumn('LatUpdate', sym_drop(df.Lat))\
    .withColumn('LonUpdate', sym_drop(df.Lon))
df_1.show(5)

+----------------+-------+--------+------+---------+---------+
|       Date/Time|    Lat|     Lon|  Base|LatUpdate|LonUpdate|
+----------------+-------+--------+------+---------+---------+
|9/1/2014 0:01:00|40.2201|-74.0021|B02512|  40.2201| -74.0021|
|9/1/2014 0:01:00|  40.75|-74.0027|B02512|    40.75| -74.0027|
|9/1/2014 0:03:00|40.7559|-73.9864|B02512|  40.7559| -73.9864|
|9/1/2014 0:06:00| 40.745|-73.9889|B02512|   40.745| -73.9889|
|9/1/2014 0:11:00|40.8145|-73.9444|B02512|  40.8145| -73.9444|
+----------------+-------+--------+------+---------+---------+
only showing top 5 rows



In [0]:
# df_dq1 = df.filter(df.Lat.like("%%"))
# df_dq1 = df.filter(df.Lat.like("%\"%") | (df.Lat.like("%\'%")) | (df.Lat.like("% %")) |
# (df.Lat.like("%,%")))
# df_dq1.show()

#UDF works GOOD 
df_dq1 = df.filter(df.Lat.like("%\"%") | (df.Lat.like("%\'%")) | (df.Lat.like("% %")) |
(df.Lat.like("%,%")))
df_dq1.show()

+---------+---+---+----+
|Date/Time|Lat|Lon|Base|
+---------+---+---+----+
+---------+---+---+----+



In [0]:
df_clean = df.exceptAll(df_dq1)

In [0]:
df_clean = df_clean.withColumn("Date/Time", F.to_timestamp(F.col("Date/Time"),"M/d/yyyy H:mm:ss")) \
                   .select(F.col('Date/Time').alias('tstamp'),
                           F.col('Lat').cast('double').alias('Lat'),
                           F.col('Lon').cast('double').alias('Lon'),
                           F.col('Base').cast('string').alias('Base'))
df_clean.printSchema()

root
 |-- tstamp: timestamp (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Lon: double (nullable = true)
 |-- Base: string (nullable = true)



In [0]:
df_clean.select(F.min(F.col('tstamp')).alias('min_tstamp'),
                F.max(F.col('tstamp')).alias('max_tstamp')).show()

+-------------------+-------------------+
|         min_tstamp|         max_tstamp|
+-------------------+-------------------+
|2014-04-01 00:00:00|2014-09-30 22:59:00|
+-------------------+-------------------+



In [0]:
# 32 A
df_base = df_clean.select(F.col('Base'), 
                          F.concat_ws('/', F.month(F.col('tstamp')),
                          F.year(F.col('tstamp'))).alias('ym'))\
                  .groupBy(F.col('Base'), F.col('ym'))\
                  .agg(F.count(F.col('Base')).alias('number_of_rides'))\
                  .orderBy(F.col('Base'), F.col('ym'), ascending = False)

df_base.show(10)

df_base.createOrReplaceTempView('agg_uber_nyc_month_rides')

+------+------+---------------+
|  Base|    ym|number_of_rides|
+------+------+---------------+
|B02764|9/2014|         178333|
|B02764|8/2014|          48591|
|B02764|7/2014|           8589|
|B02764|6/2014|           8974|
|B02764|5/2014|           9504|
|B02764|4/2014|           9908|
|B02682|9/2014|         197138|
|B02682|8/2014|         173280|
|B02682|7/2014|         196754|
|B02682|6/2014|         194926|
+------+------+---------------+
only showing top 10 rows



In [0]:
agg_blob_container = dbutils.secrets.get(scope="scope_cgbigdatalab01",key="key1")
storage_account_name = dbutils.secrets.get(scope="scope_cgbigdatalab01",key="key2")

In [0]:
dbutils.fs.mount(
    source = "wasbs://" + agg_blob_container + "@" + storage_account_name + ".blob.core.windows.net",
    mount_point = "/mnt/aggregated",
    extra_configs = {'fs.azure.account.key.' + storage_account_name + '.blob.core.windows.net':
        dbutils.secrets.get(scope = "scope_cgbigdatalab01", key = "storageAccount01key")})

Out[127]: True

In [0]:
%sql

CREATE EXTERNAL TABLE agg_uber_nyc_month_rides
USING PARQUET
LOCATION '/mnt/aggregated/agg_uber_nyc_month_rides/'
TBLPROPERTIES('serialization.null.format'='', 'skip.header.line.count'='1')
AS
select *
from agg_uber_nyc_month_rides

In [0]:
%sql
CREATE TABLE happy_hours
USING PARQUET
LOCATION '/mnt/aggregated/happy_hours/'
TBLPROPERTIES('serialization.null.format'='', 'skip.header.line.count'='1')
AS
with cte as (
select Uber_base, 
       date_format(`df_agg`.`date_time_dt`, 'hh') as happy_hours, 
       count(1) as cnt_rides,
       rank () over(order by count(1)) as rnk
from df_agg
group by 1, 2)
-- order by rn 
-- limit 3 

select *
from cte 
where rnk <= 3

In [0]:
%sql
select *
from happy_hours
limit 5

Uber_base,happy_hours,cnt_rides,rnk
Unter,01,12161,1
Unter,12,12558,2
Unter,02,12684,3


In [0]:
# for using SQL 
df_clean.createOrReplaceTempView("temp_nyc_uber")

In [0]:
%sql
select base, 
       concat(year(tstamp), '/', month(tstamp)) as ym,
       count(1) as number_of_rides 
from temp_nyc_uber
group by base, ym
order by base, ym

base,ym,number_of_rides
B02512,2014/4,35536
B02512,2014/5,36765
B02512,2014/6,32509
B02512,2014/7,35021
B02512,2014/8,31472
B02512,2014/9,34370
B02598,2014/4,183263
B02598,2014/5,260549
B02598,2014/6,242975
B02598,2014/7,245597


In [0]:
%sql
CREATE EXTERNAL TABLE e_nyc_uber
(tstamp timestamp,
 Lat double,
 Lon double,
 Base string
)       
USING CSV
OPTIONS (path "dbfs:/mnt/home/nyc_uber/",
        delimiter ",",
        header "true");

In [0]:
%sql 
select * 
from e_nyc_uber
LIMIT 5

tstamp,Lat,Lon,Base
null,40.2201,-74.0021,B02512
null,40.75,-74.0027,B02512
null,40.7559,-73.9864,B02512
null,40.745,-73.9889,B02512
null,40.8145,-73.9444,B02512


In [0]:
%sh
uber='/dbfs/mnt/home/nyc_uber'
nyc_uber_cleansed='/dbfs/mnt/home/nyc_uber_cleansed'
mkdir "$nyc_uber_cleansed"
cd $uber
ls
for file in `ls | grep 14.csv` ; do cat $file | sed -e 's/\"//g' > $nyc_uber_cleansed/clean_$file ; head -n3 $nyc_uber_cleansed/clean_$file ; done

uber-raw-data-apr14.csv
uber-raw-data-aug14.csv
uber-raw-data-jul14.csv
uber-raw-data-jun14.csv
uber-raw-data-may14.csv
uber-raw-data-sep14.csv
Date/Time,Lat,Lon,Base
4/1/2014 0:11:00,40.769,-73.9549,B02512
4/1/2014 0:17:00,40.7267,-74.0345,B02512
Date/Time,Lat,Lon,Base
8/1/2014 0:03:00,40.7366,-73.9906,B02512
8/1/2014 0:09:00,40.726,-73.9918,B02512
Date/Time,Lat,Lon,Base
7/1/2014 0:03:00,40.7586,-73.9706,B02512
7/1/2014 0:05:00,40.7605,-73.9994,B02512
Date/Time,Lat,Lon,Base
6/1/2014 0:00:00,40.7293,-73.992,B02512
6/1/2014 0:01:00,40.7131,-74.0097,B02512
Date/Time,Lat,Lon,Base
5/1/2014 0:02:00,40.7521,-73.9914,B02512
5/1/2014 0:06:00,40.6965,-73.9715,B02512
Date/Time,Lat,Lon,Base
9/1/2014 0:01:00,40.2201,-74.0021,B02512
9/1/2014 0:01:00,40.75,-74.0027,B02512


In [0]:
%sql 
ALTER TABLE e_nyc_uber 
SET LOCATION '/mnt/home/nyc_uber_cleansed';
SELECT * FROM e_nyc_uber;

tstamp,Lat,Lon,Base
null,40.2201,-74.0021,B02512
null,40.75,-74.0027,B02512
null,40.7559,-73.9864,B02512
null,40.745,-73.9889,B02512
null,40.8145,-73.9444,B02512
null,40.6735,-73.9918,B02512
null,40.7471,-73.6472,B02512
null,40.6613,-74.2691,B02512
null,40.3745,-73.9999,B02512
null,40.7633,-73.9773,B02512


In [0]:
%sql
--create external table with string type of tstamp field
drop table if exists e_nyc_uber;
CREATE EXTERNAL TABLE e_nyc_uber
(tstamp string,
 Lat double,
 Lon double,
 Base string
)       
USING CSV
OPTIONS (path "dbfs:/mnt/home/nyc_uber/",
        delimiter ",",
        header "true");

In [0]:
%sql
CREATE EXTERNAL TABLE Table_mapped
USING CSV
OPTIONS (path "dbfs:/mnt/home/nyc_uber_cleansed2/",
        delimiter ",",
        header "true")
AS
SELECT tstamp, Lat, Lon, Base, CASE
WHEN Base = 'B02512' THEN 'Unter'
WHEN Base = 'B02598' THEN 'Hinter'
WHEN Base = 'B02617' THEN 'Weiter'
WHEN Base = 'B02682' THEN 'Schmecken'
WHEN Base = 'B02764' THEN 'Danach-NY'
WHEN Base = 'B02765' THEN 'Grun'
WHEN Base = 'B02835' THEN 'Dreist'
WHEN Base = 'B02836' THEN 'Drinnen'
END AS Uber_bases
FROM e_nyc_uber;

In [0]:
%sql
SELECT tstamp, Lat, Lon, Base 
FROM e_nyc_uber
WHERE tstamp = "Date/Time"
limit 10

tstamp,Lat,Lon,Base


In [0]:
%sql
-- cause I used the header in the previous task, my headers are correctly displayed 
CREATE EXTERNAL TABLE Table_w_header
USING CSV
OPTIONS (path "dbfs:/mnt/home/nyc_uber_cleansed3/",
        delimiter ",",
        header "true")
AS
SELECT tstamp, Lat, Lon, Base 
FROM e_nyc_uber
WHERE tstamp != "Date/Time";

In [0]:
%sql
CREATE EXTERNAL TABLE Table_wo_header2
USING CSV
OPTIONS (path "dbfs:/mnt/home/nyc_uber_cleansed_ver2/",
        delimiter ",",
        header "true")
AS
SELECT tstamp, Lat, Lon, Base, ROW_NUMBER() OVER (PARTITION BY Base, tstamp, Lat, Lon ORDER BY Base) row_num
FROM e_nyc_uber
WHERE tstamp != "Date/Time";

In [0]:
%sql
SELECT *
FROM Table_wo_header2
WHERE row_num = 1
LIMIT 10

tstamp,Lat,Lon,Base,row_num
4/1/2014 0:11:00,40.769,-73.9549,B02512,1
4/1/2014 10:01:00,40.7768,-73.9773,B02512,1
4/1/2014 10:04:00,40.7741,-73.8726,B02512,1
4/1/2014 10:15:00,40.7764,-73.9598,B02512,1
4/1/2014 10:18:00,40.7745,-73.9638,B02512,1
4/1/2014 10:27:00,40.7605,-73.9991,B02512,1
4/1/2014 10:34:00,40.7226,-74.0004,B02512,1
4/1/2014 10:34:00,40.7718,-73.9654,B02512,1
4/1/2014 10:36:00,40.775,-73.9595,B02512,1
4/1/2014 10:37:00,40.7743,-73.9847,B02512,1


In [0]:
%sql
CREATE EXTERNAL TABLE Table_w_weekdays
USING CSV
OPTIONS (path "dbfs:/mnt/home/nyc_uber_cleansed_ver3/",
        delimiter ",",
        header "true")
AS
SELECT tstamp, Lat, Lon, Base, dayofweek(to_timestamp(tstamp, 'M/d/yyyy H:mm:ss')) AS weekday 
FROM e_nyc_uber
WHERE tstamp != "Date/Time";

In [0]:
%sql
-- 30 B
select to_timestamp(tstamp, 'M/d/yyyy H:mm:ss') as datetime, 
       date(to_timestamp(tstamp, 'M/d/yyyy H:mm:ss')) as date, 
       date_format(to_timestamp(tstamp, 'M/d/yyyy H:mm:ss'), 'H:mm:ss') as time,
       current_timestamp() as lastmodified
from e_nyc_uber
where tstamp != 'Date/Time'
limit 5

datetime,date,time,lastmodified
2014-09-01T00:01:00.000+0000,2014-09-01,0:01:00,2023-02-03T10:59:04.499+0000
2014-09-01T00:01:00.000+0000,2014-09-01,0:01:00,2023-02-03T10:59:04.499+0000
2014-09-01T00:03:00.000+0000,2014-09-01,0:03:00,2023-02-03T10:59:04.499+0000
2014-09-01T00:06:00.000+0000,2014-09-01,0:06:00,2023-02-03T10:59:04.499+0000
2014-09-01T00:11:00.000+0000,2014-09-01,0:11:00,2023-02-03T10:59:04.499+0000


In [0]:
cur_blob_container = dbutils.secrets.get(scope="scope_cgbigdatalab01",key="key3")
storage_account_name = dbutils.secrets.get(scope="scope_cgbigdatalab01",key="key4")

In [0]:
dbutils.fs.mount(
    source = "wasbs://" + cur_blob_container + "@" + storage_account_name + ".blob.core.windows.net",
    mount_point = "/mnt/curated",
    extra_configs = {'fs.azure.account.key.' + storage_account_name + '.blob.core.windows.net':
        dbutils.secrets.get(scope = "scope_cgbigdatalab01", key = "storageAccount01key")})

Out[147]: True

In [0]:
cur_df = spark.sql(
            "select to_timestamp(tstamp, 'M/d/yyyy H:mm:ss') as datetime, date(to_timestamp(tstamp, 'M/d/yyyy H:mm:ss')) as date, date_format(to_timestamp(tstamp, 'M/d/yyyy H:mm:ss'), 'H:mm:ss') as time, current_timestamp() as lastmodified from e_nyc_uber where tstamp != 'Date/Time'")

cur_df.show(5)

+-------------------+----------+-------+--------------------+
|           datetime|      date|   time|        lastmodified|
+-------------------+----------+-------+--------------------+
|2014-09-01 00:01:00|2014-09-01|0:01:00|2023-02-03 11:02:...|
|2014-09-01 00:01:00|2014-09-01|0:01:00|2023-02-03 11:02:...|
|2014-09-01 00:03:00|2014-09-01|0:03:00|2023-02-03 11:02:...|
|2014-09-01 00:06:00|2014-09-01|0:06:00|2023-02-03 11:02:...|
|2014-09-01 00:11:00|2014-09-01|0:11:00|2023-02-03 11:02:...|
+-------------------+----------+-------+--------------------+
only showing top 5 rows



In [0]:
cur_df.createOrReplaceTempView("cur_temp_nyc_uber")

In [0]:
%sql
CREATE EXTERNAL TABLE m_nyc_taxi
USING PARQUET
LOCATION '/mnt/curated/curated_data'
TBLPROPERTIES('serialization.null.format'='', 'skip.header.line.count'='1')
AS
SELECT * 
FROM cur_temp_nyc_uber

In [0]:
%sql
CACHE TABLE m_nyc_taxi
SELECT *
FROM m_nyc_taxi;